In [12]:
#formato binario
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import operator


In [2]:
class I_O_Datos_GoogleDrive:
    def __init__(self,scope,creds_file,GDriveName):
        self._scope=scope
        self._creds_file=creds_file
        self._GDriveName=GDriveName
        
    def obtener_datos(self):
        # use creds to create a client to interact with the Google Drive API
        #scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        scope = self._scope
        #creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        creds = ServiceAccountCredentials.from_json_keyfile_name(self._creds_file, scope)
        
        client = gspread.authorize(creds)
        # Find a workbook by name and open the first sheet
        # Make sure you use the right name here.
        sheet = client.open(self._GDriveName).sheet1

        # Extract and print all of the values
        list_of_hashes = sheet.get_all_records()
        #print(list_of_hashes
        tabla = pd.DataFrame(data=sheet.get_all_records())
        return tabla
    def transfor_datos(self,tabla):
        #almacenar en un diccionario de diccionarios cuya clave sea el usuario, y como valor un conjunto de  diccionarios
        #(clave=nombre asignatura, valor=ponderacion). No introducimos el Token ni la fecha en la que se realizó el cuestionario. 
        asignaturas= list(tabla.columns)
        diccAsi={}
        diccAsi.setdefault(1,{})
        for k in asignaturas:
            if k != 'Token'and k!='Submitted At':
                for i, j in zip(tabla[k], range(len(tabla)) ):
                    diccAsi.setdefault(j,{})
                    diccAsi[j][k]=i
        tabla1 = pd.DataFrame.from_dict(diccAsi)
        tabla1= tabla1.T
        #transformamos los vacíos en NaN para evitar el error de calcular el coeficiente de correlación de pearson entre un int y un string ('' equivale a un str)
        tabla1= tabla1.replace('', np.nan, regex=True)
        return tabla1

In [3]:
Datos_GoogleDrive=I_O_Datos_GoogleDrive(
    ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    ,'client_secret.json'
    ,"DatosTFG_SistemasRecomendacion")
Datos_GoogleDrive_Tabla=Datos_GoogleDrive.obtener_datos()
Datos_GoogleDrive_Tabla_transfor= Datos_GoogleDrive.transfor_datos(Datos_GoogleDrive_Tabla)


In [4]:
class I_O_Datos_Binario:
    def __init__(self,nombreArchivo):
        self._nombreArchivo=nombreArchivo
    #formato binario
    def obtener_datos(self):
        archivo = open(self._nombreArchivo, "rb")
        tablaDatos = pickle.load(archivo)
        archivo.close()
        return tablaDatos
    
    def guardarDatos(self, tabla1):
        archivo = open(self._nombreArchivo, "wb")
        pickle.dump(tabla1, archivo)
        archivo.close()
        
    def transfor_datos(self,tabla):
        #almacenar en un diccionario de diccionarios cuya clave sea el usuario, y como valor un conjunto de  diccionarios
        #(clave=nombre asignatura, valor=ponderacion). No introducimos el Token ni la fecha en la que se realizó el cuestionario. 
        asignaturas= list(tabla.columns)
        diccAsi={}
        diccAsi.setdefault(1,{})
        for k in asignaturas:
            if k != 'Token'and k!='Submitted At':
                for i, j in zip(tabla[k], range(len(tabla)) ):
                    diccAsi.setdefault(j,{})
                    diccAsi[j][k]=i
        tabla1 = pd.DataFrame.from_dict(diccAsi)
        tabla1= tabla1.T
        #transformamos los vacíos en NaN para evitar el error de calcular el coeficiente de correlación de pearson entre un int y un string ('' equivale a un str)
        tabla1= tabla1.replace('', np.nan, regex=True)
        return tabla1

In [5]:
IO_DATOS=I_O_Datos_Binario('archivoDatos.bin')
tabla=IO_DATOS.obtener_datos()
tabla=IO_DATOS.transfor_datos(tabla)
#IO_DATOS.guardarDatos(DATOS)


In [6]:
class Distancias:
    def coef_corr_pearson(self,dataframe,a,b,rows=True):
        dataframe
        if rows:
            common_ratings= dataframe.ix[[a,b]].dropna(axis=1).as_matrix()
            a_average = dataframe.ix[a].mean()
            b_average = dataframe.ix[b].mean()       
        else:
            common_ratings= dataframe[[a,b]].dropna(axis=0).as_matrix().T
            a_average=dataframe[a].mean()
            b_average=dataframe[b].mean()              
        if common_ratings.any(): 
            a_common_ratings=common_ratings[0,:]
            b_common_ratings=common_ratings[1,:]
            pcorr = np.dot(a_common_ratings-a_average,b_common_ratings-b_average) / ( 
                     np.sqrt(np.dot(a_common_ratings-a_average,a_common_ratings-a_average))*
                     np.sqrt(np.dot(b_common_ratings-b_average,b_common_ratings-b_average)))
            if np.isnan(pcorr):
                return 0
            else:
                return pcorr
        else:
            return 0

In [7]:
class Matriz_Similitud:
    def __init__(self,similarity):
        self._similarity=similarity
    def similarity_matrix_for_items(self,tabla, rows= False):
        # Nacho: utilizo nombres de lista generales para que pueda servir para calcular
        # las matriz de similitudes de usuarios o de asignaturas

        matrizSim={}

        if rows==True:
            lista=tabla.index.tolist()
        else:
            lista=tabla.columns.tolist()

        for x in lista:
            otras= lista[lista.index(x): len(lista)]
            matrizSim[x]= dict([(x2, self._similarity(tabla, x, x2, rows)) for x2 in otras])
        for x in lista:
            otras= lista[lista.index(x): len(lista)]
            for x2 in otras:
                matrizSim[x2][x]= matrizSim[x][x2]
        return matrizSim


In [13]:
class Filtro_Basado_Usuarios:
    def __init__(self,tabla,similitud):
        self._tabla=tabla
        self.similitud=similitud
        self.matriz_similitud=Matriz_Similitud(self.similitud)
        self.matrizSim=pd.DataFrame(self.matriz_similitud.similarity_matrix_for_items(self._tabla.T))
        self.asinaturas_cuarto=[
            #primer semestre
            'Diseño e Implementación de Sistemas Digitales',
            'Gestión de la Información',
            'Diseño y Mantenimiento del Software',
            'Organización y Gestión de Empresas',
            'Mantenimiento de Equipos Informáticos',
            'Control por Computador',
            'Hardware de Aplicación Específica',
            'Validación y Pruebas',
            'Computación Neuronal y Evolutiva',
            'Programación de Sistemas Operativos',
            #segundo semestre
            'Sistemas Distribuidos',
            'Sistemas Empotrados y de Tiempo Real',
            'Métodos Formales',
            'Nuevas Tecnologías y Empresa',
            'Minería de Datos',
            'Desarrollo Avanzado de Sistemas Software'
        ]
        
    def calculaPrediccion(self,asign,usuario):
        # Encontrar los k usuarios que evaluaron 'asign'
        #Nacho guardamos las keys de los usuarios
        kUsuarios=self._tabla[self._tabla[asign]>0].index #np.where(dataframe[asign] > 0)[0]

        # Calcular la similitud de esos k usuarios a 'usuario'
        k_Similitu=self.matrizSim[usuario][kUsuarios]

        # Unir índices de usuarios y similitudes
        k_zip=list(zip(k_Similitu,kUsuarios))

        # Media valoraciones de usuario (sobre asignaturas cursadas)
        #Nacho: la función mean() no tiene en cuenta los valores NaN, no es necesario filtrar por valores >0
        user_u_average=self._tabla.T[usuario].mean() #[list(np.where(dataframe.T[usuario]>0)[0])].mean()

        # Media de los valores absolutos de similitud de vecions próximos a usuario
        k_sim=[k_zip[i][0] for i in range(len(k_zip))]
        k_sim_average=np.abs(np.array(k_sim)).sum()

        # Estimación del rating a la asignatura asign
        rating_user_u_item_i=0
        for user_v in [k_zip[i][1] for i in range(len(k_zip))]:
            rating_user_u_item_i+= (self._tabla.T[user_v][asign] - self._tabla.T[user_v].mean())*self.matrizSim[usuario][user_v]
        rating_user_u_item_i=user_u_average + rating_user_u_item_i/k_sim_average
        #Devolvemos el rating estimado
        return rating_user_u_item_i
    def calcula_Prediccion_Cuarto(self,usuario):
        for i in self.asinaturas_cuarto:
            yield (i,self.calculaPrediccion(i,usuario))
            

In [14]:
distancias=Distancias()
filtro_Basado_Usuarios=Filtro_Basado_Usuarios(tabla,distancias.coef_corr_pearson)


In [15]:
usuario=2
predic=dict(filtro_Basado_Usuarios.calcula_Prediccion_Cuarto(usuario))
predic = sorted(predic.items(), key=operator.itemgetter(1),reverse=True)
predic

[('Control por Computador', 4.296633133198905),
 ('Validación y Pruebas', 4.133610485844622),
 ('Minería de Datos', 3.9502678618405507),
 ('Diseño y Mantenimiento del Software', 3.9407602856983424),
 ('Hardware de Aplicación Específica', 3.7851875131110067),
 ('Desarrollo Avanzado de Sistemas Software', 3.736321496385728),
 ('Diseño e Implementación de Sistemas Digitales', 3.714416716357596),
 ('Computación Neuronal y Evolutiva', 3.608691499241023),
 ('Programación de Sistemas Operativos', 3.5828854168009996),
 ('Sistemas Distribuidos', 3.53261502259667),
 ('Organización y Gestión de Empresas', 3.2591297298454056),
 ('Sistemas Empotrados y de Tiempo Real', 3.185253700590988),
 ('Métodos Formales', 3.1112223044195737),
 ('Nuevas Tecnologías y Empresa', 3.109305563503443),
 ('Gestión de la Información', 3.0812529463513467),
 ('Mantenimiento de Equipos Informáticos', 2.8803380183989824)]

# guardar y vaciar asignaturas

In [ ]:
def vaciarAsignaturas( asignaturas):
    cont=1
    for i in list(asignaturas.keys()):
        asignaturas[i] = 0
        cont+=1
    return asignaturas

In [ ]:
asignaturas= {}

In [ ]:
def guardarAsignaturas( diccion, asigna, valor):
    asignaturas[asigna.replace('_', " ", regex=True)] = valor
    return asignaturas

In [ ]:
asignaturas= guardarAsignaturas(  asignaturas, 'Análisis_y_Diseño_de_Sistemas', 4)

In [ ]:
asignaturas

In [ ]:

asignaturas=vaciarAsignaturas( asignaturas)

In [ ]:
asignaturas

In [ ]:
def funDiccSem1(values):
    diccSem1 = {}
    diccSem1['Fundamentos Deontológicos y Jurídicos de las TIC']= values[0]
    diccSem1['Álgebra Lineal']= values[1]
    diccSem1['Informática Básica']= values[2]
    diccSem1['Fundamentos Físicos de la Informática']= values[3]
    diccSem1['Matemática Discreta']= values[4]
    return diccSem1

In [ ]:
def funDiccSem2(values):
    diccSem2 = {}
    diccSem2['Inglés Aplicado a la Informática']= values[0]
    diccSem2['Cálculo']= values[1]
    diccSem2['Programación']= values[2]
    diccSem2['Fundamentos de Computadores']= values[3]
    diccSem2['Sistemas Operativos']= values[4]
    return diccSem2

In [ ]:
def funDiccSem3(values):
    diccSem3 = {}
    diccSem3['Metodología de la Programación']= values[0]
    diccSem3['Estadística']= values[1]
    diccSem3['Ingeniería del Software']= values[2]
    diccSem3['Bases de Datos']= values[3]
    diccSem3['Arquitectura de Computadores']= values[4]
    return diccSem3

In [ ]:
def funDiccSem4(values):
    diccSem4 = {}
    diccSem4['Estructuras de Datos']= values[0]
    diccSem4['Redes']= values[1]
    diccSem4['Interacción Hombre-Máquina']= values[2]
    diccSem4['Fundamentos de Organización y Gestión de Empresas']= values[3]
    diccSem4['Análisis y Diseño de Sistemas']= values[4]
    return diccSem4

In [ ]:
def funDiccSem5(values):
    diccSem5 = {}
    diccSem5['Arquitecturas Paralelas']= values[0]
    diccSem5['Sistemas Inteligentes']= values[1]
    diccSem5['Gestión de Proyectos']= values[2]
    diccSem5['Diseño y Administración de Sistemas y Redes ']= values[3]
    diccSem5[ 'Procesadores del Lenguaje']= values[4]

In [ ]:
def funDiccSem6(values):
    diccSem6 = {}
    diccSem6['Programación Concurrente y de Tiempo Real']= values[0]
    diccSem6['Seguridad Informática']= values[1]
    diccSem6['Aplicaciones de Bases de Datos']= values[2]
    diccSem6['Algoritmia']= values[3]
    diccSem6['Métodos Numéricos y Optimización']= values[4]

In [ ]:
def funDiccSem7(values):
    diccSem7 = {}
    diccSem7['Diseño e Implementación de Sistemas Digitales']= values[0]
    diccSem7['Gestión de la Información']= values[1]
    diccSem7['Diseño y Mantenimiento del Software']= values[2]
    diccSem7['Organización y Gestión de Empresas']= values[3]
    diccSem7['Mantenimiento de Equipos Informáticos']= values[4]    
    diccSem7['Control por Computador']= values[5]
    diccSem7['Hardware de Aplicación Específica']= values[6]
    diccSem7['Validación y Pruebas']= values[7]
    diccSem7['Computación Neuronal y Evolutiva']= values[8]
    diccSem7['Programación de Sistemas Operativos']= values[9]
    return diccSem7


In [ ]:
def funDiccSem8(values):
    diccSem8 = {}
    diccSem8['Sistemas Distribuidos']= values[0]
    diccSem8['Sistemas Empotrados y de Tiempo Real']= values[1]
    diccSem8['Métodos Formales']= values[2]
    diccSem8['Nuevas Tecnologías y Empresa']= values[3]
    diccSem8['Minería de Datos']= values[4]
    diccSem8['Desarrollo Avanzado de Sistemas Software']= values[5]
    return diccSem8

In [ ]:
def porDefecto(asignaturas, dataframe):
    for i in dataframe.columns:
        asignaturas[i]=3
    return asignaturas

In [ ]:
asi= porDefecto(asignaturas, dataframe)

In [ ]:
asi

In [ ]:
# guardar valoración semestre